## 可视化工具(tensorboard)
tensorflow不仅仅是一个计算图软件，其还包含了tensorboard可视化工具，安装tensorflow的时候会默认安装，使用方法非常简单，使用writer = tf.summary.FileWriter('./graph', sess.graph)就能够创建一个文件写入器，./graph是存储目录，sess.graph表示读入的图结构。

In [1]:
import tensorflow as tf
a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
with tf.Session() as sess:
    writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(x))
writer.close()  # close the writer when you’re done using it

5


然后打开终端，运行程序，接着输入`tensorboard --logdir="./graphs"`，然后点击提示信息中的网站，就能够进入tensorboard，可以得到下面的结果。
![tensorboard](https://pic2.zhimg.com/80/v2-f326d404c78795e13828efa053dd17b5_hd.png)


## 常数类型(Constant types)
- 能够通过下面这个方式创造一个常数
- tf.constant(value, dtype=None, shape=None, name='Const', verify_shape=False)

In [2]:
a = tf.constant([2,2], name='a')
b = tf.constant([[0,1], [2,3]], name='b')
x = tf.multiply(a, b, name='dot_production')
with tf.Session() as sess:
    print(sess.run(x))

[[0 2]
 [4 6]]


这跟numpy里面的是差不多的，同时还有一些特殊值的常量创建。

In [11]:
shape = [2,2]
sess = tf.Session()
# zeros()创建shape.x * shape.y的矩阵
print(sess.run(tf.zeros(shape, dtype=tf.float32, name=None)))
tensor=[[2,4,4],[1,3,6]]

# zeros_like()创建全0矩阵，形状和tensor一样
print(sess.run(tf.zeros_like(tensor, dtype=None, name=None, optimize=True)))
print(sess.run(tf.ones(shape, dtype=tf.float32, name=None)))
print(sess.run(tf.ones_like(tensor, dtype=None, name=None, optimize=True)))


[[ 0.  0.]
 [ 0.  0.]]
[[0 0 0]
 [0 0 0]]
[[ 1.  1.]
 [ 1.  1.]]
[[1 1 1]
 [1 1 1]]
[[8 8 8]
 [8 8 8]]


In [12]:
# tf.fill(dims, value, name=None) --用value填充形状为dims的张量
# 参数说明:
# dims: 输出张量的形状
# value: 填充返回的张量的值
print(sess.run(tf.fill([2, 3], 8)))

[[8 8 8]
 [8 8 8]]


In [13]:
# tf.linspace(start, stop, num, name=None)
print(sess.run(tf.linspace(10.0, 13.0, 4)))

[ 10.  11.  12.  13.]


In [16]:
# tf.range(start, limit=None, delta=1, dtype=None, name='range')
print(sess.run(tf.range(3, limit=18, delta=3)))

[ 3  6  9 12 15 18]


### 除此之外还可以产生一些随机数
```
# 正态分布随机数，均值mean,标准差stddev
tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
# 截断正态分布随机数，均值mean,标准差stddev,不过只保留[mean-2stddev,mean+2stddev]范围内的随机数
tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None,
name=None)
# 均匀分布随机数，范围为[minval,maxval]
tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None,
name=None)
# 沿着张量value的第一维进行随机重新排列
tf.random_shuffle(value, seed=None, name=None)
# 随机地将张量value裁剪为给定的大小
tf.random_crop(value, size, seed=None, name=None)
# 从多项式分布logits中抽取num_samples个样本.
tf.multinomial(logits, num_samples, seed=None, name=None)
# 从每个给定的 Gamma distribution(s) 中绘制 shape 样本.
tf.random_gamma(shape, alpha, beta=None, dtype=tf.float32, seed=None, name=None)
```
- 例如：
```
samples = tf.random_gamma([10], [0.5, 1.5]) 
# samples 的形状为[10, 2], 其中每个 slice [:, 0] 和 [:, 1] 表示从每个分布中抽取的样本
samples = tf.random_gamma([7, 5], [0.5, 1.5]) 
# samples 形状为[7, 5, 2], 其中每个 slice [:, :, 0] 和 [:, :, 1] 表示从两个分布中的每一个中抽取 7x5 个样本
samples = tf.random_gamma([30], [[1.],[3.],[5.]], beta=[[3., 4.]]) 
# samples 形状为 [30, 3, 2], 每个 3x2 分布有30个样本
```

另外tensorflow和numpy的数据类型可以通用，也就是说

In [18]:
import numpy as np
tf.ones([2, 2], np.float32)

<tf.Tensor 'ones_2:0' shape=(2, 2) dtype=float32>

#### 最好不要使用python自带的数据类型，同时在使用numpy数据类型的时候要小心，因为未来可能tensorflow的数据类型和numpy不再兼容。

## 变量(Variable)
使用常量会存在什么问题呢？常量会存在计算图的定义当中，如果常量过多，这会使得加载计算图变得非常慢，同时常量的值不可改变，所以引入了变量。

In [24]:
a = tf.Variable(2, name='scalar')
b = tf.Variable([2, 3], name='vector')
c = tf.Variable([[0, 1], [2, 3]], name='matrix')
w = tf.Variable(tf.zeros([784, 10]), name='weight')

变量有着下面几个操作
```
x = tf.Variable()
x.initializer # 初始化
x.eval() # 读取里面的值
x.assign() # 分配值给这个变量
```

注意一点，在使用变量之前必须对其进行初始化，初始化可以看作是一种变量的分配值操作。最简单的初始化方式是一次性初始化所有的变量

In [26]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

也可以对某一部分变量进行初始化

In [27]:
init_ab = tf.variables_initializer([a, b], name='init_ab')
with tf.Session() as sess:
    sess.run(init_ab)

None


或者是对某一个变量进行初始化

In [28]:
w = tf.Variable(tf.zeros([784, 10]))
with tf.Session() as sess:
    sess.run(w.initializer)

如果我们想取出变量的值，有两种方法

In [29]:
w = tf.Variable(tf.truncated_normal([10, 10], name='normal'))
with tf.Session() as sess:
    sess.run(w.initializer)
    print(w.eval()) # 方法一
    print(sess.run(w)) # 方法二

[[-0.95718139 -0.44025454  0.74425966  0.90783101  1.92811644 -1.23942387
  -0.5437609  -0.38250542 -0.25278467  0.53650796]
 [-0.20639697  0.09634554 -0.97924918  0.65590286 -0.59741789  0.72083384
   0.08172197  0.11697232 -0.12097789 -0.59242022]
 [ 1.69721115  1.22957861 -0.51178795  0.7769385   1.58861959  0.45306814
  -0.86427718  0.64466608  1.5474869  -1.05312073]
 [-0.39219511  0.49143392  0.35518053 -0.87263775  1.71564329  0.91315299
   0.67178315 -0.64709389 -0.51290148  0.63789964]
 [-1.39125645 -0.33293372 -0.28284857 -0.07524792  0.42671156 -0.89730573
  -0.95670503  0.27647999 -0.58710933  1.34452248]
 [ 0.69753832  0.85837597 -1.39729309  0.55970442 -1.2486279  -0.82782286
  -0.59798783  0.90180582  0.61606443  1.78436077]
 [ 0.94495797  0.13594955 -0.51260149 -0.47672337 -0.60010576  0.64251441
  -0.27183887 -1.1965642  -1.3056109   0.22318153]
 [ 1.31254125  0.29588953 -1.42722034  1.03387058  1.98050785 -0.73330659
  -0.72953737  0.37646601  0.17473647 -0.49377769]


下面看看这个小程序

In [30]:
w = tf.Variable(10)
w.assign(100)
with tf.Session() as sess:
    sess.run(w.initializer)
    print(w.eval())

10


上面这个程度会得到10,这是因为我们虽然定义了assign操作，但是tensorflow是在session中执行操作，所以我们需要执行assign操作。

In [31]:
w = tf.Variable(10)
assign_op = w.assign(100)
with tf.Session() as sess:
    sess.run(w.initializer)
    sess.run(assign_op)
    print(w.eval())

100


另外tensorflow的每个session是相互独立的，我们可以看看下面这个例子

In [32]:
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()
sess1.run(W.initializer)
sess2.run(W.initializer)
print(sess1.run(W.assign_add(10))) # >> 20
print(sess2.run(W.assign_sub(2))) # >> 8
print(sess1.run(W.assign_add(100))) # >> 120
print(sess2.run(W.assign_sub(50))) # >> -42
sess1.close()
sess2.close()

20
8
120
-42


你也可以根据一个变量来定义一个变量

In [ ]:
w = tf.Variable(tf.truncated_normal([700, 10]))
u = tf.Variable(w * 2)

## 占位符(Placeholders)
tensorflow中一般有两步，第一步是定义图，第二步是在session中进行图中的计算。对于图中我们暂时不知道值的量，我们可以定义为占位符，之后再用`feed_dict`去赋值。

定义占位符的方式非常简单
```
tf.placeholder(dtype, shape=None, name=None)
```
dtype是必须要指定的参数，shape如果是None，说明任何大小的tensor都能够接受，使用shape=None很容易定义好图，但是在debug的时候这将成为噩梦，所以最好是指定好shape。

In [33]:
a = tf.placeholder(tf.float32, shape=[3])
b = tf.constant([5, 5, 5], tf.float32)
c = a + b
with tf.Session() as sess:
    print(sess.run(c, feed_dict={a: [1, 2, 3]}))

[ 6.  7.  8.]


除此之外，也可以给tensorflow中的运算进行feed操作，如下

In [36]:
a = tf.add(2, 3)
b = tf.multiply(a, 3)
with tf.Session() as sess:
    print(sess.run(b))
    print(sess.run(b, feed_dict={a: 2}))

15
6


### lazy loading
lazy loading是指你推迟变量的创建直到你必须要使用他的时候。下面我们看看一般的loading和lazy loading的区别。

In [37]:
# normal loading
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(z)

# lazy loading
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(tf.add(x, y))

normal loading 会在图中创建x和y变量，同时创建x+y的运算，而lazy loading只会创建x和y两个变量。这不是一个bug，那么问题在哪里呢？

normal loading在session中不管做多少次x+y，只需要执行z定义的加法操作就可以了，而lazy loading在session中每进行一次x+y，就会在图中创建一个加法操作，如果进行1000次x+y的运算，normal loading的计算图没有任何变化，而lazy loading的计算图会多1000个节点，每个节点都表示x+y的操作。

看到了吗，这就是lazy loading造成的问题，这会严重影响图的读入速度。